In [258]:
import pandas as pd
import numpy as np
import re
import string
import os
import csv
import math
pd.set_option('display.max_colwidth', -1)

In [377]:
def read_file(file_path):
    df = pd.read_csv(file_path, header=None, encoding="utf-8")
    df.columns = ['lines']   
    title = df.lines[0]
    ep_num = re.sub(r'[^0-9]', '', file_path)
    df = df[df['lines'] != '\n']
 
    # General line clean-up:
    df['lines'] = df['lines'].apply(lambda x: re.sub('', "'", x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('…', '', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('\n', ' ', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('�', "'", x))
    df['lines'] = df['lines'].apply(lambda x: x.lower().lstrip())
    
    
    # Handle colons that appear within lines of dialogue
    df['lines'] = df['lines'].apply(lambda x: re.sub(r'\d[:]', '', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('hint:', 'hint', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('note:', 'note', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('here:', 'here', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('like:', 'like', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('flashback to:', 'flashback to', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('be:', 'be', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('lapse:', 'lapse', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('is:', 'is', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('it:', 'it', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('at:', 'at', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('said:', 'said', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('listen:', 'listen', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('also:', 'also', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('look:', 'look', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('says:', 'says', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('that:', 'that', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('thought:', 'thought', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('sung:', 'sung', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('me:', 'me', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('line:', 'line', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('one:', 'one', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('continues:', 'continues', x))
    df['lines'] = df['lines'].apply(lambda x: re.sub('\):', '\)', x))
    
    
    # Get rid of transcriber notes
    df['lines'] = df['lines'].apply(lambda x: re.sub(r'([\{]).*?([\}])', '', x))
    
    
    df = df[df['lines'] != '']
    df = df.reset_index(drop=True)
    
    cleaned_lines = []
    
    for i in range(len(df)):
        #print(i)
        if df.lines[i][0] == ':':
            cleaned_lines[-1] += df.lines[i]
        else:
            cleaned_lines.append(df.lines[i])
    
    df = pd.DataFrame(columns=['lines'])
    
    df['lines'] = cleaned_lines
    
    ep_scene = []
    ep_character = []
    ep_dialogue = []
    
    current_scene = None
    current_character = None
    current_dialogue = None
    
    if ep_num == 210:
        current_scene
    
    df = df.reset_index(drop=True)
    
    
    #print(df[:50])

    start_point = 0
    
    
    # Handle problematic transcripts
    
    if ep_num == 203:
        start_point = 6
        current_scene = 'Unknown'
        
    elif ep_num == 210:
        start_point = 5
        current_scene = 'newsstand'
    
    
    
    for i in range(start_point, len(df)):
        #print(df.lines[i])
        if 'scene:' in df.lines[i]:
            current_scene = re.sub(r'[\[\]]', '', re.sub('scene: ', '', df.lines[i]))
        elif 'scene,' in df.lines[i]:
            current_scene = re.sub(r'[\[\]]', '', re.sub('scene, ', '', df.lines[i]))
        elif 'scene :' in df.lines[i]:
            current_scene = re.sub(r'[\[\]]', '', re.sub('scene :', '', df.lines[i]))
        
        elif df.lines[i] in ['end', 'opening credits', 'commercial break', 'closing credits', 'opening titles']:
                current_scene = df.lines[i]
                ep_scene.append(current_scene)
                current_character = None
                current_dialogue = None
                ep_character.append(current_character)
                ep_dialogue.append(current_dialogue)
        
        elif ':' in df.lines[i] and (current_scene != None and current_character == None): # and df.lines[i].split(':', 1)[1][0] not in ['0','1','2','3','4','5','6','7','8','9']):
            if df.lines[i].rstrip().split(':', 1)[1] != '':
                current_character = df.lines[i].split(':', 1)[0]
                current_dialogue = df.lines[i].split(':', 1)[1]
            else: 
                current_character = df.lines[i].split(':', 1)[0]
        
        elif current_character != None and current_scene != None:         
            
            if current_dialogue == None: 
                current_dialogue = df.lines[i]
            else:
                current_dialogue += ' ' + df.lines[i]      
        
        # Need to handle dialogue that is split across multpile lines
        if i < (len(df)-1):
            if ':' in df.lines[i+1] or df.lines[i+1] in ['end', 'opening credits', 'commercial break', 'closing credits', 'opening titles']:                
                if current_scene != None and (current_character != None and current_dialogue != None):
                    ep_scene.append(current_scene)
                    ep_character.append(current_character)
                    ep_dialogue.append(current_dialogue)
                    current_character = None
                    current_dialogue = None
                    
    # Clean lines
    
    ep_dialogue = [re.sub(r'([\[]).*?([\]])', '', x) if x != None else x for x in ep_dialogue]
    ep_dialogue = [re.sub(r'([\(]).*?([\)])', '', x) if x != None else x for x in ep_dialogue]
    ep_dialogue = [re.sub(r'[^a-z ]', '', x) if x != None else x for x in ep_dialogue]
    
    # One transcriber used character codes in lieu of full character names. Map those to the correct names
    
    ep_character = ['chandler' if x == 'chan' else x for x in ep_character]
    ep_character = ['rachel' if x == 'rach' else x for x in ep_character]
    ep_character = ['phoebe' if x == 'phoe' else x for x in ep_character]
    ep_character = ['monica' if x == 'mnca' else x for x in ep_character]
    ep_character = ['fun bobby' if x == 'fbob' else x for x in ep_character]
    ep_character = ['estelle' if x == 'estl' else x for x in ep_character]
    
    # Correct missplelings
    
    ep_character = ['woman' if x == 'an' else x for x in ep_character]
    ep_character = ['rachel' if x == 'racel' else x for x in ep_character]
    ep_character = ["joey's voice on answering machine" if x == "(machine--joey's voice)" else x for x in ep_character]
    ep_character = ['woman' if x == 'an' else x for x in ep_character]
    
    
    
    
    def chop_scene(scn):
        comma_place = scn.find(',')
        period_place = scn.find('.')
        if comma_place == -1 and period_place == -1:
            return scn
        elif comma_place < period_place and comma_place != -1:
            return scn.split(',')[0]
        else:
            return scn.split('.')[0]
           
    ep_scene = [chop_scene(x) for x in ep_scene]
 
    # Problematic episodes:
    # 210 - scene descriptions leave scene location at end of phrase; there is a scene description split across two lines
    # 908 - first scene after opening credits should be ross and rachels apartment. it apperas as unlabeled text without a "scene" label in the transcript
    return_df = pd.DataFrame(columns=['scene', 'character', 'dialogue'])
    return_df['scene'] = ep_scene
    return_df['character'] = ep_character
    return_df['dialogue'] = ep_dialogue
    return_df['ep_title'] = title
    return_df['ep_num'] = ep_num  
    return_df = return_df[return_df['dialogue'] != '']
    return_df = return_df.reset_index(drop=True)
    
    return return_df

In [367]:
test = read_file('friendsscraper/spiders/ep-210.csv')


                                                                                                                                                                                                  lines
0   the one with russ                                                                                                                                                                                  
1   written by ira ungerlieder.                                                                                                                                                                        
2   transcribed by josh hodge.                                                                                                                                                                         
3   minor additions and adjustments by dan silverstein.                                                                                                                                                


In [378]:
episode_dfs = []

for episode in os.listdir('friendsscraper/spiders/'):
    if 'ep-' in episode: 
        path = 'friendsscraper/spiders/' + episode
        episode_dfs.append(read_file(path))
        
        #df = process_script(raw_lines, ep_title)
        #ep_title = ep_title.lower()
        #df.to_csv('friendsscraper/spiders/processed_ep_%s' % ep_title, index=False)

In [379]:
big_friends = episode_dfs[0]

for ep in episode_dfs[1:]:
    big_friends = pd.concat([big_friends, ep])

In [285]:
len(big_friends)

61742

In [380]:
uni = big_friends['character'].unique()
for i in uni:
    print(i)

chandler
rachel
phoebe
monica
all
None
joey
ross
julie
mich
waiter
guy
gunther
receptionist
carl
the doctor
janice
the instructor
janice's voice
the potential roommate
parker
mrs. geller
mr. geller
woman
man
aunt lisa
uncle dan
emily
devon
liam
ticket counter attendant
mrs. chatracus
rachel and bonnie
bonnie
phoebe sr.
ursula
chandler, monica, and joey
ross and rachel
phoebe and rachel
phoebe, ross, and rachel
joshua
mr. waltham
the a.d
the cigarette smoking guy
charlton heston
the cigarette guy
the interviewer
host
mike
lauren
both
jill
janine
stephanie
karin
meg
russell
commercial voiceover
rob
erica
lipson
kids
rachel and monica
janitor
tv doctor
joey on tv
kid
security guard
trainer
director's assistant
susie
monica and rachel
director
terry
boys
doctor
girl 1 on bus
girl 2 on bus
chandler and joey
transit authority guy
together
fun bobby
russ
estelle
gang
stranger
duncan
rahcel
old woman
the fan
buffay, the vampire layer
the vampire
man's voice
another man's voice
same man's voice

In [381]:
big_friends[big_friends['character'] == "no problem! next week"]

,scene,character,dialogue,ep_title,ep_num
249,chandler is packing the broken china in its box,no problem! next week,stealing,the one with rachel's other sister,908


In [385]:
big_friends[big_friends['ep_num'] == "908"][100:150]

,scene,character,dialogue,ep_title,ep_num
100,opening credits,amy,oh thats a funny noise monica and amy turn away and walk into the living room towards the secret closet,the one with rachel's other sister,908
101,opening credits,joey,pheebs i still need some help here phoebe oh right ok um ok so its not just the lie you tell but its the way you tell it for example if you look down at the ground when youre talking people know youre lying,the one with rachel's other sister,908
102,opening credits,joey,oh i dont know why this is so hard for me you know i mean lying is basically just acting and i am a terrfic actor,the one with rachel's other sister,908
103,opening credits,phoebe looks down,you are a terrific actor joey realizes what shes doing,the one with rachel's other sister,908
104,opening credits,amy coming out of the bathroom,hey hey wheres the baby,the one with rachel's other sister,908
105,opening credits,rachel,oh we just put her down for a nap,the one with rachel's other sister,908
106,opening credits,amy,oh i was just thinking you know what would be incredible if you guys died,the one with rachel's other sister,908
107,opening credits,ross first has a look of 'huh' then changes it to sarcastic happy,thank you amy,the one with rachel's other sister,908
108,opening credits,amy,no no then i would get the baby i mean you know it would be just like a movie like at first i wouldnt know what to do with her then i would rise to the occasion and and then i would get a makeover and then id get married phoebe thats a great movie she claps joey nods in agreement,the one with rachel's other sister,908
109,opening credits,amy,now listen not that you guys could stop me or anything cause you know youd be dead i was thinking about changing her name im just not really a big fan of emily,the one with rachel's other sister,908


In [372]:
big_friends[big_friends['ep_num'] == '108'][:50]

,scene,character,dialogue,ep_title,ep_num
0,chandler's office,shelley,hey gorgeous hows it going,the one where nana dies twice,108
1,chandler's office,chandler,dehydrated japanese noodles under fluorescent lights does it get better than this,the one where nana dies twice,108
2,chandler's office,shelley,question youre not dating anybody are you because i met somebody who would be perfect for you,the one where nana dies twice,108
3,chandler's office,chandler,ah ysee perfect might be a problem had you said codependent or selfdestructive,the one where nana dies twice,108
4,chandler's office,shelley,do you want a date saturday,the one where nana dies twice,108
5,chandler's office,chandler,yes please,the one where nana dies twice,108
6,chandler's office,shelley,okay hes cute hes funny hes,the one where nana dies twice,108
7,chandler's office,chandler,hes a he,the one where nana dies twice,108
8,chandler's office,shelley,well yeah oh god i just i thought good shelley im just gonna go flush myself down the toilet now okay goodbye,the one where nana dies twice,108
9,opening credits,None,None,the one where nana dies twice,108


In [371]:
big_friends[big_friends['dialogue'] == 'couldnt enjoy a cup of coffee after that']

,scene,character,dialogue,ep_title,ep_num
